In [ ]:
import sys
sys.path.append("/Users/user/question-retrieval-KIPerWeb/")
from utils import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import json
from nltk.stem.cistem import Cistem
from nltk.corpus import stopwords
from ranx import Qrels, evaluate, Run
import swifter
import json
import requests
from sklearn.preprocessing import LabelEncoder
from trectools import TrecPoolMaker, TrecRun
from tqdm import tqdm
from collections import Counter
import spacy
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from operator import itemgetter
import joblib
import numpy as np
import pulp
from math import log2

## Read the QB's sample

In [ ]:
# Read preprocessed data
# qb = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/samples/qb_stratified-sample.csv").fillna("N/A")
# qb = qb.drop(columns=['docid'])
# qb.rename(columns={"category":"topic_label_de_fixed"}, inplace=True)
# qb['docid'] = qb.index
# qb['content'] = qb['content'].apply(lambda x:remove_stop_words(x, 'german'))


# Preprocessing
# Models for preprocessing
nlp = spacy.load("de_core_news_sm")
tagger_de = ht.HanoverTagger('morphmodel_ger.pgz')

# qb['content'] = qb['content'].apply(lambda x : preprocess_documents(x, tagger_de))
# qb

## Calculate Embeddings and Initialize NMSLIB Indices

In [ ]:
# model_1 = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
# model_2 = SentenceTransformer("aari1995/German_Semantic_STS_V2")
# model_3 = SentenceTransformer("sentence-transformers/LaBSE")
# model_4 = SentenceTransformer("PM-AI/bi-encoder_msmarco_bert-base_german")
# model_5 = SentenceTransformer("efederici/e5-base-multilingual-4096")
# model_6 = SentenceTransformer("intfloat/multilingual-e5-base")
# model_7 = SentenceTransformer("clips/mfaq")
# model_8 = SentenceTransformer("PM-AI/sts_paraphrase_xlm-roberta-base_de-en")
# model_9 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-euclidean")
# model_10 = SentenceTransformer("LLukas22/all-MiniLM-L12-v2-embedding-all")
# model_11 = SentenceTransformer("LLukas22/paraphrase-multilingual-mpnet-base-v2-embedding-all")
# model_12 = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v1")
# model_13 = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
# model_14 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-cosine")
# model_15 = SentenceTransformer("shibing624/text2vec-base-multilingual")
# model_16 = SentenceTransformer("Sahajtomar/German-semantic")
# model_17 = SentenceTransformer("setu4993/LaBSE")
# model_18 = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")
# model_19 = SentenceTransformer("and-effect/musterdatenkatalog_clf")
# model_20 = SentenceTransformer("nblokker/debatenet-2-cat")
# model_21 = SentenceTransformer("setu4993/LEALLA-large")
# model_22 = SentenceTransformer("dell-research-harvard/lt-wikidata-comp-de")
# model_23 = SentenceTransformer("ef-zulla/e5-multi-sml-torch")
# model_24 = SentenceTransformer("barisaydin/text2vec-base-multilingual")
# model_25 = SentenceTransformer("meta-llama/Llama-2-7b-chat-hf")

In [ ]:
# # # # # Create the indices

# # # index_1 = create_index(qb, model_1, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-paraphrase-multilingual-mpnet-base-v2.nmslib")
# # # index_2 = create_index(qb, model_2, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/aari1995-German_Semantic_STS_V2.nmslib")
# # # index_3 = create_index(qb, model_3, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-LaBSE.nmslib")
# # # index_4 = create_index(qb, model_4, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/PM-AI-bi-encoder_msmarco_bert-base_german.nmslib")
# # # index_5 = create_index(qb, model_5, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/efederici-e5-base-multilingual-4096.nmslib")
# # # index_6 = create_index(qb, model_6, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/intfloat-multilingual-e5-base.nmslib")
# # # index_7 = create_index(qb, model_7, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/clips-mfaq.nmslib")
# # # index_8 = create_index(qb, model_8, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/PM-AI-sts_paraphrase_xlm-roberta-base_de-en.nmslib")
# # # index_9 = create_index(qb, model_9, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/deutsche-telekom-gbert-large-paraphrase-euclidean.nmslib")
# # # index_10 = create_index(qb, model_10, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/LLukas22-all-MiniLM-L12-v2-embedding-all.nmslib")
# # # index_11 = create_index(qb, model_11, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/LLukas22-paraphrase-multilingual-mpnet-base-v2-embedding-all.nmslib")
# # # index_12 = create_index(qb, model_12, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-distiluse-base-multilingual-cased-v1.nmslib")
# # # index_13 = create_index(qb, model_13, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-distiluse-base-multilingual-cased-v2.nmslib")
# # # index_14 = create_index(qb, model_14, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")
# # # index_15 = create_index(qb, model_15, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/shibing624-text2vec-base-multilingual.nmslib")
# # # index_16 = create_index(qb, model_16, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/Sahajtomar-German-semantic.nmslib")
# # # index_17 = create_index(qb, model_17, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/setu4993-LaBSE.nmslib")
# # # index_18 = create_index(qb, model_18, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/symanto-sn-xlm-roberta-base-snli-mnli-anli-xnli.nmslib")
# # # index_19 = create_index(qb, model_19, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/and-effect-musterdatenkatalog_clf.nmslib")
# # # index_20 = create_index(qb, model_20, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/nblokker-debatenet-2-cat.nmslib")
# # # index_21 = create_index(qb, model_21, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/setu4993-LEALLA-large.nmslib")
# # # index_22 = create_index(qb, model_22, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/dell-research-harvard-lt-wikidata-comp-de.nmslib")
# # # index_23 = create_index(qb, model_23, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/ef-zulla-e5-multi-sml-torch.nmslib")
# # # index_24 = create_index(qb, model_24, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/barisaydin-text2vec-base-multilingual.nmslib")
# # # index_25 = create_index(qb, model_25, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/meta-llama-Llama-2-7b-chat-hf.nmslib")


# # # # Or load them:

# index_1 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-paraphrase-multilingual-mpnet-base-v2.nmslib")
# index_2 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/aari1995-German_Semantic_STS_V2.nmslib")
# index_3 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-LaBSE.nmslib")
# index_4 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/PM-AI-bi-encoder_msmarco_bert-base_german.nmslib")
# index_5 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/efederici-e5-base-multilingual-4096.nmslib")
# index_6 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/intfloat-multilingual-e5-base.nmslib")
# index_7 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/clips-mfaq.nmslib")
# index_8 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/PM-AI-sts_paraphrase_xlm-roberta-base_de-en.nmslib")
# index_9 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/deutsche-telekom-gbert-large-paraphrase-euclidean.nmslib")
# index_10 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/LLukas22-all-MiniLM-L12-v2-embedding-all.nmslib")
# index_11 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/LLukas22-paraphrase-multilingual-mpnet-base-v2-embedding-all.nmslib")
# index_12 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-distiluse-base-multilingual-cased-v1.nmslib")
# index_13 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-distiluse-base-multilingual-cased-v2.nmslib")
# index_14 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")
# index_15 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/shibing624-text2vec-base-multilingual.nmslib")
# index_16 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/Sahajtomar-German-semantic.nmslib")
# index_17 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/setu4993-LaBSE.nmslib")
# index_18 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/symanto-sn-xlm-roberta-base-snli-mnli-anli-xnli.nmslib")
# index_19 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/and-effect-musterdatenkatalog_clf.nmslib")
# index_20 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/nblokker-debatenet-2-cat.nmslib")
# index_21 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/setu4993-LEALLA-large.nmslib")
# index_22 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/dell-research-harvard-lt-wikidata-comp-de.nmslib")
# index_23 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/ef-zulla-e5-multi-sml-torch.nmslib")
# index_24 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/barisaydin-text2vec-base-multilingual.nmslib")
# index_25 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/meta-llama-Llama-2-7b-chat-hf.nmslib")


## Run Queries Against Search Indices

In [ ]:
# query_test = search('Gesundheitswesen'.lower(), index_25, model_25, qb, k=100)['search_output']
# query_test

In [ ]:
## We could use queries:
# queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/topics_experiments/topics/topics.csv")
queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/queries/queries.csv")
queries_file['queries'] = queries_file['queries'].apply(clean_text)



print("-> Number of queries:",len(queries_file['queries'].tolist()))

queries = queries_file['queries'].tolist()
query_ids = queries_file['qid'].tolist()


In [ ]:


def maximize_ndcg(data):
    """
    Function to maximize the NDCG score for a given list of tuples (docid, query, document, relevance_score).

    Parameters:
    data (list of tuples): A list containing tuples in the format (docid, query, document, relevance_score).

    Returns:
    dict: A dictionary with the query as the key and a list of tuples ordered to maximize NDCG as the value.
    """

    # Create a dictionary of documents and their scores for each query
    doc_scores = {}
    for docid, query, document, score in data:
        if query not in doc_scores:
            doc_scores[query] = {}
        doc_scores[query][document] = (docid, score)

    # Initialize the MILP model
    model = pulp.LpProblem("Maximize_NDCG", pulp.LpMaximize)

    # Decision variables: binary variables for each document and rank position
    x = {}
    for query in doc_scores:
        num_docs = len(doc_scores[query])
        for doc, _ in doc_scores[query].items():
            for rank in range(1, num_docs + 1):
                x[(doc, rank)] = pulp.LpVariable(f"x_{doc}_{rank}", 0, 1, pulp.LpBinary)

    # Objective function: Maximize DCG
    dcg = pulp.lpSum([x[(doc, rank)] * doc_scores[query][doc][1] / log2(rank + 1)
                      for query in doc_scores
                      for doc in doc_scores[query]
                      for rank in range(1, len(doc_scores[query]) + 1)])
    model += dcg

    # Constraints
    for query in doc_scores:
        num_docs = len(doc_scores[query])
        
        # Each document is assigned to exactly one position
        for doc in doc_scores[query]:
            model += pulp.lpSum([x[(doc, rank)] for rank in range(1, num_docs + 1)]) == 1

        # Each position has exactly one document
        for rank in range(1, num_docs + 1):
            model += pulp.lpSum([x[(doc, rank)] for doc in doc_scores[query]]) == 1

    # Solve the model using CBC solver with multi-threading
    solver = pulp.PULP_CBC_CMD(threads=11)  # None lets CBC decide how many threads to use
    model.solve(solver=solver)

    # Extract the optimal ordering of documents
    optimal_ordering = {}
    for query in doc_scores:
        sorted_docs = sorted(doc_scores[query].keys(), key=lambda doc: sum(rank * x[(doc, rank)].varValue for rank in range(1, len(doc_scores[query]) + 1)))
        optimal_ordering[query] = [doc_scores[query][doc] for doc in sorted_docs]

    return optimal_ordering

# # Example usage
data = [(5, 'digitaler schutz', 'fallen leicht sache konzentrieren', -0.079115465), (29, 'digitaler schutz', 'suchen neu arbeitsstelle möchten informieren firma nähe wohnort frage kommen googeln', -0.007935083), (45, 'digitaler schutz', 'gewerblich schutzrecht schützen erscheinungsbild produkt designrecht geschmacksmusterrecht nennen schützen erscheinungsbild produkt einschließlich form farbe textur', 0.29081506), (46, 'digitaler schutz', 'analysiert stärken schwächen chance risiko unternehmen beeinflussen swotanalys analyse', 0.0109088365), (51, 'digitaler schutz', 'markenname name symbol produkt dienstleistung markt identifizieren rechtlich schützen', 0.008363422), (54, 'digitaler schutz', 'art schutz bieten urheberrecht schutz unbefugt vervielfältigung kreativ werk', 0.2860862), (87, 'digitaler schutz', 'ziel metallurgisch qualitätskontrolle materialfestigkeit', 0.098113), (92, 'digitaler schutz', 'wichtig komponente erfolgreich teamkommunikation offenheit transparenz', 0.06391461), (95, 'digitaler schutz', 'fähigkeit verhandlung besonders wichtig geduld', -0.030299678), (96, 'digitaler schutz', 'ziel marketing kundenzufriedenheit bindung umsatzsteigerung markenbekanntheit positionierung', 0.043777116), (106, 'digitaler schutz', 'professionelle melden telefon bestehen bestimmt komponente der reihenfolge einhalten ordnen zeitlich abfolge', 0.11474205), (109, 'digitaler schutz', 'contentmarketing bereitstellung relevant inhalt kundenbindung', 0.0834216), (111, 'digitaler schutz', 'ziel markenpositionierung bewusst platzierung mark verhältnis mark kopf verbraucher', 0.10543488), (115, 'digitaler schutz', 'verstehen begriff branding schaffen pflegen einzigartig identität wiedererkennung produkt mark', 0.10031161), (120, 'digitaler schutz', 'telemarketingüberwachung überwachung telemarketingagent qualität anrufen einhaltung vorschrift sicherstellen', 0.2609344), (135, 'digitaler schutz', 'geeignet kriterium zuverlässigkeit information internet beurteilen vergleichen mehrere relevant quelle gesucht information', 0.08803589), (139, 'digitaler schutz', 'womit zugriff cloud faxgerät', 0.21891595), (140, 'digitaler schutz', 'backup datensicherung', 0.3749509), (143, 'digitaler schutz', 'begriff stammen themenbereich pc sicherheit harmonizer', 0.3701905), (150, 'digitaler schutz', 'onlinekommunikation genauso verantwortlich ding online geben dafür straße sagen', 0.07652815), (153, 'digitaler schutz', 'möglich software applikation personalisieren ja', 0.22045097), (158, 'digitaler schutz', 'technisch nichttechnisch problem lösen wählen passend digital technologie geraten anwendung software service wende', 0.33267808), (159, 'digitaler schutz', 'sicherheitseinstellung firewall digital gerät konfigurieren weiß tun pc virus befallen werden', 0.33761752), (163, 'digitaler schutz', 'womit digital gerät markieren auswählen maus', 0.15137762), (170, 'digitaler schutz', 'name bezeichnen suchmaschine googeln', 0.019761898), (175, 'digitaler schutz', 'lage alltäglich problem nutzung digital technologie auftreten lösen programm schließen computer starten programm installieren neu stand bringen internetverbindung überprüfen', 0.33830994), (177, 'digitaler schutz', 'bewusst digital technologie schnell weiterentwickeln deshalb versuchen wissenslücke schließen bringen digital fähigkeit regelmäßig neu stand', 0.3288284), (179, 'digitaler schutz', 'eingabetast markieren', 0.12974294), (197, 'digitaler schutz', 'warum wichtig zeit präsentation auge behalten mehrere antwort möglich sicherstellen geplant inhalt behandeln präsentation pünktlich enden', 0.10650341), (199, 'digitaler schutz', 'infografik regel visuell darstellung information datum', 0.14180517), (226, 'digitaler schutz', 'schreiben präposition jeweilig frei feld', 0.11687661), 
        (247, 'digitaler schutz', 'begriff bezeichnen ort datum speichern cloud', 0.14151599), (272, 'digitaler schutz', 'lagerung handeln umformungsaufgabe umfüllen mischen aussortieren gut', 0.04203859), (275, 'digitaler schutz', 'sichtkontrolle machen schaden verpackung ware entdecken dokument brauchen transportschaden dokumentieren dokumentation machen begleitpapier lieferschein', 0.25390935), (281, 'digitaler schutz', 'bedeuten begreifen inventur bezug lagermanagement zählung überprüfung lagerbestand', 0.166396), (285, 'digitaler schutz', 'benutzung fördermittel müssen bestimmt sicherheitshinweis einhalten verhalten arbeiter korrekt', 0.24169931),
        (294, 'digitaler schutz', 'zweck lagersicherheitssystem sicherung lager lagerbestand', 0.3783385), 
        (297, 'digitaler schutz', 'bringen tätigkeit abfallentsorgung reihenfolge umwelt bestmöglich schützen wiederverwenden', 0.13370971), 
        (299, 'digitaler schutz', 'lagerprinzip lassen paternosterlager anwenden lifo fifoprinzip', 0.07839871), (304, 'digitaler schutz', 'hauptziel lagerhaltung unternehmen sicherstellung verfügbarkeit produkt', 0.24935073), (311, 'digitaler schutz', 'vorteil einsatz gabelstaplern lager effizient transport', 0.046194963), 
        (340, 'digitaler schutz', 'ss abs ss abs hgb vermögensgegenständ verbindlichkeit unabhängig voneinander bewerten methode abweichung grundsatz einzelbewertung dabei gesamtverfahren', 0.034971707), (341, 'digitaler schutz', 'vermögenswert inventur erfassen vorrat', 0.0074666813), (344, 'digitaler schutz', 
        'gestaltungskriterium sicherung liquidität entwicklung working capital', 0.09081703),
        (348, 'digitaler schutz', 'prinzip inventarerstellung zählen vollständigkeit', 0.15288809), (400, 'digitaler schutz', 'verstehen begriff netikette verhaltensregeln kommunikation internet', 0.02175406), (410, 'digitaler schutz', 'internet produkt bewerten achten darauf bewertung begründen', -0.07263721), (417, 'digitaler schutz', 'acht darauf online kommunizieren sogenannt netiquette einhalten', 0.23935978),
        (419, 'digitaler schutz', 'internetbrowser funktion seite merken registerkaren verwalten internetseite angeben benutzen', 0.22729494), (420, 'digitaler schutz', 'halten auge offen neu anwendung finden alltagsproblem lösen', 0.11022307), (421, 'digitaler schutz', 'verschieden forum sozial netzwerk häufig anonym nickname einzig öffentlich sichtbar identifikationsmerkmal mitgliederwi versuchen dennoch angemessen respektvoll kommunikation sicher stellen geben netiquett einhalten moderator überwachen', -0.059226543), (423, 'digitaler schutz', 'datei inhalt online person austauschen speziell programm nutzen', 0.10211474), (424, 'digitaler schutz', 'sozial netzwerk nutzen einstellung privatsphär', -0.1354462), 
        (425, 'digitaler schutz', 'weiß mails der datenanhängen verschlüsseln', 0.1155689), (428, 'digitaler schutz', 'wissen digital technologie sicher benutzen dienstleistungsbetrieb bank krankenhaus interagieren', 0.27956343), (429, 'digitaler schutz', 'wissen zuverlässigkeit glaubwürdigkeit informationsquell internet feststellen', 0.13154766), (431, 'digitaler schutz', 'funktion online services verwenden egovernment onlineshopping achten dabei sicherheit datum', 0.19897076), (434, 'digitaler schutz', 'onlinekommunikation genauso verantwortlich ding online geben dafür straße sagen', 0.07652815), (436, 'digitaler schutz', 'internet mensch interagieren machen stets bewusst kulturell hintergrund', 0.07905684), (440, 'digitaler schutz', 'bedeuten seo search engine optimization optimierung webseit suchmaschine gut finden', -0.08650622),
        (441, 'digitaler schutz', 'messengerdienst enthalten zusätzlich sicherheitshürde', 0.30413604), (447, 'digitaler schutz', 'liegen technisch fehler smartphone computer fallen leicht problem dahinter herausfinden', 0.17414644), (448, 'digitaler schutz', 'bei onlinebanking abhängig geldinstitut folgend dienstleistung anbieten kontostand prüfen', 0.05163823), (512, 'digitaler schutz', 'leistungsindikator managementkontrollsystem messbar groß leistung fortschreiten unternehmen quantifizieren', 0.09232404), (516, 'digitaler schutz', 'intern kontrolle managementkontrollsystem', 0.3078072), (517, 'digitaler schutz', 'warum regelmäßig überprüfung anpassung managementkontrollsystem wichtig', 0.17226797), (526, 'digitaler schutz', 'schlüsselindikator managementkontrollsystem messbar groß leistung unternehmen wichtig bereich verfolgen', 0.22584844), (528, 'digitaler schutz', 'umfassen steuerung managementkontrollsyst anpassung aktivität ressource zielerreichung sicherstellen', 0.15713789), (529, 'digitaler schutz', 'zentral element gut managementkontrollsystem', 0.051601823), (533, 'digitaler schutz', 'tool verwaltung zeit helfen zutreffend antwort taschenkalend email projektmanagementsoftware', 0.13286543), (540, 'digitaler schutz', 'rolle spielen balanced scorecard managementkontrollsystem', 0.17719036), (550, 'digitaler schutz', 'steuerungsprinzip arbeiten aufgeführt vorrichtung relais', 0.028483087), (554, 'digitaler schutz', 'bedeuten zeichen schalten verboten', 0.18767647), (555, 'digitaler schutz', 'messe widerstehen isolation motor ähnlich videoder motor arbeiten v spannung stellen messgerät v', 0.072229564), (557, 'digitaler schutz', 'hauptzweck sicherungsautomat elektrisch schaltung stromflus begrenzen überlastung schützen', 0.2520886), (565, 'digitaler schutz', 'elektrisch schaltung', 0.23168299), (569, 'digitaler schutz', 'schaltungsart handeln wechselschaltung', 0.16064292), (577, 'digitaler schutz', 'geraten leitfähig gehäuseteil schutzleitersystem verbunden ordnen schutzklasse schutzklasse i', 0.29714778), (578, 'digitaler schutz', 'schalter schließen verbraucher bauteil geraten unten gerade nummer', 0.07480959), (580, 'digitaler schutz', 'video sehen klemmverbindung herstellen werden zange dafür', 0.053095773), (582, 'digitaler schutz', 'elektrische geraten berührung fremdkörp wasserschutz kennzeichnen klasse', 0.20487995), (584, 'digitaler schutz', 'gehen innenwiderstand internal resistance messgeräten strommesser sollen möglichst klein innenwiderstand spannungsmesser sollen möglichst klein innenwiderstand', 0.0947056), (585, 'digitaler schutz', 'sehen klemmverbindungist ausgeführt ja', -0.060616203), (659, 'digitaler schutz', 'gehören informationspflicht onlineshop forum', 0.1589899), (677, 'digitaler schutz', 'lesen nachfolgend auszug jugendschutzgesetz genau handeln hierbei ausschneiden jeweilig gesetzestext prüfen anschließend getroffen aussage entscheiden aussage nummerjährig nummernummer uhr dienst beenden beginnen nächst morgen nummernumm uhr beendigung täglich arbeitszeit dürfen jugendlich ablauf ununterbrochen freizeit mind std beschäftigen', 0.2129688), (680, 'digitaler schutz', 'tätigkeit beschreiben dienstleistung installateur bei einsetzen dichtung', 0.12144931), (692, 'digitaler schutz', 'ordnen unten stehend ziel jeweilig kategorie sicherung existenzminimum', 0.1478532), (700, 'digitaler schutz', 'lesen nachfolgend auszug jugendschutzgesetz genau handeln hierbei ausschneiden jeweilig gesetzestext prüfen anschließend getroffen aussage entscheiden aussage gaststätte krankenhaus dürfen jugendlich zwei wochenende monat beschäftigen', 0.18110085), (718, 'digitaler schutz', 'konsum produktivgüter ordnen jeweils korrekt rohstoff', -0.0085335625), (724, 'digitaler schutz', 'gezeigt maschine anlage müssen regelmäßig warten tätigkeit gehören bereich wartung abschalten', 0.06469712), (729, 'digitaler schutz', 'bohren arbeiter bohrung kontrollieren durchmesser bohrung mm betragen abweichung mm liegen toleranzbereich kontrolle missen arbeiter wert mmwas tun entsorgen produkt produzieren', 0.0797507), (732, 'digitaler schutz', 'video stellen arbeiter schraubverbindung her dargestellt verbindung ausführen', 0.08096566), (736, 'digitaler schutz', 'bedeuten zeichen betreten verbieten', 0.13928123), (748, 'digitaler schutz', 'bedeuten zeichen warnung magnetisch feld', 0.07453702), (757, 'digitaler schutz', 'warum arbeiter messtoleranz einhalten mitarbeiter lernen genau arbeiten', 0.018783035), (766, 'digitaler schutz', 'häufig verwendet werkzeug verwaltung projekt aufgabenmanagementsoftwar', 0.082880564), (778, 'digitaler schutz', 'aktivität form zeitverschwendens betrachtet übermäßig surfen internet', 0.096285336), (792, 'digitaler schutz', 'rolle spielen klar ziel aufgabenplanung helfen dabei fokus behalten', 0.01698084), (796, 'digitaler schutz', 'tool aufgabenverwaltung hilfreich todolisten', 0.029893745), (803, 'digitaler schutz', 'kriterium priorisierung wichtig zutreffend antwort dringlichkei komplexität persönlich vorlieben', 0.021354543), (807, 'digitaler schutz', 'merkmal effektiv todolist klar priorisierung', 0.04591178), (810, 'digitaler schutz', 'vorteil verwendung aufgabenmanagementsoftwar effizienzsteigerung', 0.07504825), (826, 'digitaler schutz', 'verstehen kundendienst vertreiben unterstützung betreuung kunde kauf', 0.05890409), (844, 'digitaler schutz', 'vertriebskanal nutzen folgend produkt dienstleistung produkt a produkt d', -0.031877533), (848, 'digitaler schutz', 'vertriebskanal wichtig erzielung neukunde vergleich kundenbindung neukundengewinnung kundenbindung', 0.020249186), (867, 'digitaler schutz', 'bauwerk bauwerk brauchen gut holzschutz wichtig sicherheit regel bauwerk brauchen gut holzschutz', 0.16662276), (882, 'digitaler schutz', 'holz sollen holz schützen sollen chemisch holzschutzmittel benutzen benutzen chemisch holzschutzmittel art holz schlecht chemisch holzschutzmittel bearbeiten', 0.20478421), (884, 'digitaler schutz', 'brett eichen brettern mm breit brettern feuchtigkeit feuchtigkeit ändern ändern volumen feuchtigkeit steigen breit brett mm', -0.010313265), (902, 'digitaler schutz', 'glas ceilingeffeken bezug frau unternehmen unsichtbar barriere frau daran hindern hoch führungsposition aufsteigen', 0.09296177), (949, 'digitaler schutz', 'bild suchfilt markieren', 0.20547898), (955, 'digitaler schutz', 'beurteilen aussage datei cloud speichern abrufen brauchen internetzugang ja immer', 0.124918625), (960, 'digitaler schutz', 'fehlend datum datenvorverarbeitung datenwert datensatz tabelle fehlen verfügbar', 0.18506366), (965, 'digitaler schutz', 'verstehen datenbereinigung datenvorverarbeitung datenbereinigung beziehen prozeß identifizierung korrektur fehler ungenauigkeit fehlend werten rohdaten', 0.1756526), (970, 'digitaler schutz', 'verstehen datenvorverarbeitung bereinigung transformation aggregation rohdat analyse modellierung vorbereiten', 0.04951571), (974, 'digitaler schutz', 'verstehen hoax internet bewußt verbreitet falschinformation', 0.11269541), (977, 'digitaler schutz', 'verstehen datenvorverarbeitung vorbereitung rohdat analyse', 0.1162215), (984, 'digitaler schutz', 'gängig browser windows htmlquellcoden webseit anzeige lassen strg+u', 0.008449831), (987, 'digitaler schutz', 'möchten datum sicher verlust äußern einfluß stromschlag feuer schützenbitte markieren hierfür gut geeignet speichermöglichkeit cloud', 0.19633493), 
        (989, 'digitaler schutz', 'falschbitte beurteilen schaltfläche prüfen webseite lassen aufgerufen seite zuverlässigkeit überprüfen', 0.19910449), (994, 'digitaler schutz', 'hauptzweck vpn virtual private network vpn dienen hauptsächlich sicher verbindung unsicher netzwerk herstellen privatsphär benutzer schützen', 0.20610899), (1002, 'digitaler schutz', 'verstehen begriff virtual reality vr computergeneriert umgebung immersiv erfahrung benutzer schaffen', 0.054878686), (1004, 'digitaler schutz', 'botnet netzwerk infiziert computer fernsteuern', 0.099479586), (1006, 'digitaler schutz', 'kind gehen täglich fuß schule sicherheitsgrund möchten immer genau wissen befinden hierzu statten kind gut smartwatch trackingfunktion', 0.22057608), (1008, 'digitaler schutz', 'technologie verwaltung zeit helfen zutreffend antwort nutzung app software aufgabenverwaltung', 0.22975922), 
        (1010, 'digitaler schutz', 'trojanisch pferd bezug computer schädlich software nützlich anwendung tarnen', 0.19728811), (1011, 'digitaler schutz', 'smartphone zeigen meldung gerätespeich voll maßnahme sein sinnvoll benötigt datum bild videos löschen', 0.31277895), (1012, 'digitaler schutz', 'verstehen blockchain dezentral unveränderlich datenbanktechnologie', 0.10859331), (1018, 'digitaler schutz', 'begriff beschreiben automatisch steuerung haushaltsgeräten geraten internet ding iot smart home', 0.18999133), (1019, 'digitaler schutz', 'verstehen begriff blockchain dezentral unveränderlich datenbanktechnologie', 0.10160131), (1021, 'digitaler schutz', 'exceldatei kollege kompatibilitätsproblem rechnermit dargestellt formel geraten installiert version anzeige lassen', 0.11229286), (1028, 'digitaler schutz', 'cybersicherheit praktikum technologie schutz computersystem netzwerk sicherheitsbedrohung', 0.4242568), (1030, 'digitaler schutz', 'zweck firewall informationstechnologie firewall dienen netzwerk unerlaubt zugriff schädlich angriff schützen', 0.22218339), (1181, 'digitaler schutz', 'funktion aufsichtsrat normalerweise unternehmen überwachung beratung geschäftsführung', 0.12297936), (1193, 'digitaler schutz', 'verstehen whistleblowerschutz corporate governance whistleblowerschutz beziehen maßnahme sicherstellen person missständ unternehmen hinweisen repressalie schützen', 0.2719007), (1203, 'digitaler schutz', 'rolle spielen compliance unternehmensführung einhaltung gesetz vorschrift ethisch standard risiko minimieren vertrauen stakeholder erhalten', 0.17691776), (1208, 'digitaler schutz', 'bedeutung compliance corporate governance einhaltung gesetz vorschrift sowie ethisch ausrichtung unternehmensverhalten', 0.14769842), (1210, 'digitaler schutz', 'hauptmerkmal gut corporate governance transparenz offenlegung information', 0.11645921), (1217, 'digitaler schutz', 'gesetz vorschrift schützen recht mensch behinderung bezug beruflich rehabilitation american with disabilities act ada unbehindertenrechtskonvention', 0.1621336), (1244, 'digitaler schutz', 'beispiel technisch hilfsmittel beruflich rehabilitation einsetzen bildschirmlesegeräten sehbehindert spracherkennungssoftwar mensch eingeschränkt mobilität', 0.07038623), (1265, 'digitaler schutz', 'beschäftigen wichtig schutzgesetz betreiben aushangpflichtig gehören allgemein gleichbehandlungsgesetz', 0.32013124), (1268, 'digitaler schutz', 'verhaltensregel brandfall entstehungsphase brandes löschversuch unternehmen dabei gefahr bringen', 0.04141882), (1269, 'digitaler schutz', 'zweck sicherheitsdatenblatt sdb bezug chemikalie arbeitsplatz', 0.17674434), (1273, 'digitaler schutz', 'sicherheitsdatenblatt dokument information gefahrstoff der sicher verwendung arbeitsplatz enthalten', 0.26020086), (1274, 'digitaler schutz', 'verhalten bei löschen feuerlöscher gehen bei löschen immer wind', 0.11089953), (1275, 'digitaler schutz', 'mitarbeiter tun arbeitsunfall verletzung erleiden vorfall sofort vorgesetzt sicherheitsabteilung melden', 0.2260598), (1279, 'digitaler schutz', 'schutzmaßnahme erst linie ergreifen auswirkung chemikalie arbeitsplatz minimieren verwendung persönlich schutzausrüstung ps schutzbrill handschuh schutzanzug erst linie erforderlich auswirkung chemikalie arbeitsplatz minimieren', 0.33247417), (1284, 'digitaler schutz', 'maßnahme effektiv vermeidung stolperunfäll lagerhaus ordnungsgemäß kennzeichnung beseitigung hindernis unebenheit boden effektiv vermeidung stolperunfäll lagerhaus', 0.14880367), (1286, 'digitaler schutz', 'hauptzweck sicherheitsdatenblättern sds arbeitsplatz hauptzweck sicherheitsdatenblättern sds arbeitsplatz information sicher handhabung verwendung chemikalie bereitstellen', 0.3516069), (1289, 'digitaler schutz', 'bedeuten begreifen hazard communication bezug arbeitssicherheit', 0.2386741), (1290, 'digitaler schutz', 'erstehilfekit arbeitsplatz enthalten', 0.12083721), (1295, 'digitaler schutz', 'sollen mitarbeiter tun arbeitsunfall verletzung arbeitsplatz erleiden', 0.06594057), (1296, 'digitaler schutz', 'bedeuten begreifen ergonomie zusammenhang arbeitssicherheit', 0.16379902), (1302, 'digitaler schutz', 'bedeuten begreifen notfallpla bezug arbeitssicherheit vorbereitet plan maßnahme bewältigung notfallen arbeitsplatz festlegen', 0.20936447), (1304, 'digitaler schutz', 'wichtig voraussetzung vermeidung stolperfall arbeitsplatz regelmäßig beseitigung hindernis ordentlich arbeitsplatzgestaltung', 0.15177569), (1307, 'digitaler schutz', 'dürfen weiterverwenden copyright verletzen werk der rechteinhaber zustimmung verwendung ausdrücklich geben', 0.13500473), (1310, 'digitaler schutz', 'copyright schützen buch', 0.38969308), (1311, 'digitaler schutz', 'usergenerated content inhalt benutzer community erstellen bewertung kommentar beitrag sozial medium', -0.026048463), (1316, 'digitaler schutz', 'aspekt sollen erstellung videoinhalt onlineplattformen berücksichtigen videoqualität auflösung dauer videos', 0.12761416), (1321, 'digitaler schutz', 'information veröffentlichen möglichst breit publikum unabhängig betriebssystem geräteübergreifend verfügung stehen medium offen standardisiert format verfügung stellen', 0.132394), (1331, 'digitaler schutz', 'wichtig schritt planung inhalt websit definition zielgruppe bedürfnis', 0.041942075), (1333, 'digitaler schutz', 'verstehen begriff content marketing erstellung verbreitung nützlich relevant inhalt zielgruppe anziehen binden', 0.03993506), (1334, 'digitaler schutz', 'bedeutung einhaltung urheberrechtsbestimmung verwendung bild grafik inhalt wichtig rechtlich problem vermeiden', 0.2832985), (1335, 'digitaler schutz', 'möchten firmenpräsentation verwenden creativ commons cc kennzeichnen dürfen ja', -0.038380485), (1340, 'digitaler schutz', 'bedeuten begreifen call to action cta bezug inhaltserstellung aufforderung leser bestimmt handlung ausführen newslett abonnieren produkt kaufen weit information anmelden', 0.10838721), (1342, 'digitaler schutz', 'element sollen effektiv emailmarketingkampagn enthalten auffällig betreffzeile klar handlungsaufforderung call to action', 0.10089511), (1344, 'digitaler schutz', 'content asset wertvoll wiederverwendbar inhalt beitragen publikum binden mark unternehmen stärken', 0.10710881), (1421, 'digitaler schutz', 'wichtig ziel buchführung unternehmen finanztransparenz genauigkeit sicherstellen', 0.13033181), (1427, 'digitaler schutz', 'unterschied abschreibung amortisation abschreibung beziehen wertminderung sachanlage gebäude amortisation wertminderung immateriell vermögenswert patent lizenz betreffen', 0.08213428), (1445, 'digitaler schutz', 'verstehen phishing versuch sensibel information täuschung stehlen', 0.23474246), (1446, 'digitaler schutz', 'dosangriff denial of service bezug internetsicherheit angriff angreifer versuchen dienst websit netzwerk überlasten legitim benutzer unzugänglich', 0.3136779), (1447, 'digitaler schutz', 'malware malware schädlich software entwickeln werden computer gerät infizieren schaden anrichten', 0.31274745), (1449, 'digitaler schutz', 'phishing betrügerisch versuch vertraulich information passwört kreditkartennummer gelangen vertrauenswürdig quelle ausgeben', 0.3543419), (1450, 'digitaler schutz', 'person schützen gerät gut unberechtigt zugriff', 0.2673263), (1451, 'digitaler schutz', 'aussage treffen sicherheitsprogramm sollen regelmäßig aktualisieren', 0.29656428), (1452, 'digitaler schutz', 'sicher httpsverbindung httpsverbindung verschlüsseln verbindung sicherheit datum übertragung internet gewährleisten', 0.19588025), (1455, 'digitaler schutz', 'einfach energiesparmaßnahm pc nutzung standby ruhezustand', 0.15026766), (1457, 'digitaler schutz', 'verschlüsselung wahr wählen zutreffend antwort verschlüsselung helfen vertraulich datum unbefugt zugriff schützen', 0.2509815), (1459, 'digitaler schutz', 'maßnahme bewähren praxis sicherheit internet aktualisierung software betriebssystem regelmäßig', 0.33283302), (1460, 'digitaler schutz', 'beurteilen folgend aussagekreditkartendat internet bedenkenlos angeben mißbrauchen aufgrund international sicherheitsbestimmung ausgeschlossen aussage', 0.26659486), (1461, 'digitaler schutz', 'zweifaktorauthentifizierung 2fa sicherheitsmethode benutzer authentifizierung zwei verschieden identifikationsfaktor verwenden müssen', 0.2462319), (1463, 'digitaler schutz', 'firewall sicherheitsvorrichtung datenverkehr intern netzwerk internet überwachen kontrollieren unerlaubt zugriff bedrohung blockieren', 0.29824287), (1464, 'digitaler schutz', 'phishing betrügerisch technik benutzer täuschen persönlich information preisgeben', 0.11121318), (1466, 'digitaler schutz', 'gehen datenverbindung extern dienst aufgrund gesperrt port möglich öffnen einstellung korrigieren offensichtlich eingestellt remoteport', 0.10015268), (1469, 'digitaler schutz', 'sicherheitsmaßnahme webentwicklung wichtig httpsverschlüsselung crosssite scripting xss schutz sqlinjectionprävention', 0.28866568), (1470, 'digitaler schutz', 'firewall sicherheitsvorrichtung datenverkehr intern netzwerk internet überwacht regulieren', 0.19010378), (1471, 'digitaler schutz', 'firewalls korrekt wählen zutreffend antwort firewalls sicherheitsvorrichtung datenverkehr netzwerk kontrollieren', 0.1642974), (1472, 'digitaler schutz', 'onlinewörterbüch nutzer inhalt versorgen deshalb möglicherweise immer ganz verlässlich dictcc', -0.0011392236), (1473, 'digitaler schutz', 'ergonomische arbeiten computer empfohlenmach klein pause wenig minute dehnenlockern undoder umherlaufen minute', 0.16715515), (1474, 'digitaler schutz', 'vpn virtual private network vpn dienst internetverbindung verschlüsseln ipadressen verbirgt onlineprivatsphär schützen', 0.20188496), (1476, 'digitaler schutz', 'schutzmittel metallbearbeitung augenverletzung verhindern augenschutzbrill', 0.21726407), (1477, 'digitaler schutz', 'firewall bezug internetsicherheit sicherheitsvorrichtung software datenverkehr netzwerk internet überwachen kontrollieren unautorisiert zugriff verhindern', 0.33936256),
        (1480, 'digitaler schutz', 'email erhalten gehen denken gefälscht email handeln laden deshalb angeboten sicherheitsupdat herunter', 0.32639778), (1483, 'digitaler schutz', 'vpn virtual private network bezug internetsicherheit sicher tunnel internetverbindung verschlüsseln onlineaktivitäten anonymisieren', 0.25476003), (1484, 'digitaler schutz', 'vpn verschlüsselt internetzugang privatsphäre schützen', 0.22950402), (1485, 'digitaler schutz', 'best practices sicher surfen internet wählen zutreffend antwort verwenden komplex passwort ändern regelmäßig', 0.2299964), (1486, 'digitaler schutz', 'antivirenprogramm antivirenprogramm software entwickeln werden schädlich virus malware computer erkennen entfernen', 0.27553356), (1488, 'digitaler schutz', 'wozu dienen firewall schutz netzwerk unautorisiert zugriff', 0.18245658), (1489, 'digitaler schutz', 'maßnahme wichtig sicherheit onlinekonten gewährleisten verwendung stark passwort zweifaktorauthentifizierung', 0.34480426), (1509, 'digitaler schutz', 'körpersprache ostasien ordnen jeweilig bedeutung korrekt', -0.010752937), (1520, 'digitaler schutz', 'verstehen sogenannt 3aformel abfedern auslassen anteilnehmen', 0.0438644)]


# def maximize_ndcg_mrr(data, ndcg_weight=0.5, mrr_weight=0.5):
#     """
#     Function to maximize a weighted sum of NDCG and MRR for a given list of tuples (docid, query, document, relevance_score).

#     Parameters:
#     data (list of tuples): A list containing tuples in the format (docid, query, document, relevance_score).
#     ndcg_weight (float): The weight assigned to NDCG in the objective function.
#     mrr_weight (float): The weight assigned to MRR in the objective function.

#     Returns:
#     dict: A dictionary with the query as the key and a list of tuples ordered to maximize the weighted sum of NDCG and MRR as the value.
#     """

#     # Create a dictionary of documents and their scores for each query
#     doc_scores = {}
#     for docid, query, document, score in data:
#         if query not in doc_scores:
#             doc_scores[query] = {}
#         doc_scores[query][document] = (docid, score)

#     # Initialize the MILP model
#     model = pulp.LpProblem("Maximize_NDCG_MRR", pulp.LpMaximize)

#     # Decision variables: binary variables for each document and rank position
#     x = {}
#     for query in doc_scores:
#         num_docs = len(doc_scores[query])
#         for doc, _ in doc_scores[query].items():
#             for rank in range(1, num_docs + 1):
#                 x[(doc, rank)] = pulp.LpVariable(f"x_{doc}_{rank}", 0, 1, pulp.LpBinary)

#     # Objective function: Weighted sum of NDCG and MRR
#     ndcg = pulp.lpSum([x[(doc, rank)] * doc_scores[query][doc][1] / log2(rank + 1)
#                        for query in doc_scores
#                        for doc in doc_scores[query]
#                        for rank in range(1, len(doc_scores[query]) + 1)])
    
#     mrr = pulp.lpSum([x[(doc, 1)] / rank
#                       for query in doc_scores
#                       for doc in doc_scores[query]
#                       for rank in range(1, len(doc_scores[query]) + 1)])

#     model += ndcg_weight * ndcg + mrr_weight * mrr

#     # Constraints
#     for query in doc_scores:
#         num_docs = len(doc_scores[query])
        
#         # Each document is assigned to exactly one position
#         for doc in doc_scores[query]:
#             model += pulp.lpSum([x[(doc, rank)] for rank in range(1, num_docs + 1)]) == 1

#         # Each position has exactly one document
#         for rank in range(1, num_docs + 1):
#             model += pulp.lpSum([x[(doc, rank)] for doc in doc_scores[query]]) == 1

#     # Solve the model using CBC solver with multi-threading
#     solver = pulp.PULP_CBC_CMD(threads=None)  # None lets CBC decide how many threads to use
#     model.solve(solver=solver)

#     # Extract the optimal ordering of documents
#     optimal_ordering = {}
#     for query in doc_scores:
#         sorted_docs = sorted(doc_scores[query].keys(), key=lambda doc: sum(rank * x[(doc, rank)].varValue for rank in range(1, len(doc_scores[query]) + 1)))
#         optimal_ordering[query] = [doc_scores[query][doc] for doc in sorted_docs]

#     return optimal_ordering


# new_ordering = maximize_ndcg_mrr(data)
# new_ordering

## Generate searches and store them as JSON files

In [ ]:
# path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_CrossEncoder-Re-Ranker/json_format/'


# for i, j in zip(queries, query_ids):
#     print(i, j)
#     search(i, index_1, model_1, qb, k=100, run_path=f'{path}query_{j}/run_01.json')
#     search(i, index_2, model_2, qb, k=100, run_path=f'{path}query_{j}/run_02.json')
#     search(i, index_3, model_3, qb, k=100, run_path=f'{path}query_{j}/run_03.json')
#     search(i, index_4, model_4, qb, k=100, run_path=f'{path}query_{j}/run_04.json')
#     search(i, index_5, model_5, qb, k=100, run_path=f'{path}query_{j}/run_05.json')
#     search(i, index_6, model_6, qb, k=100, run_path=f'{path}query_{j}/run_06.json')
#     search(i, index_7, model_7, qb, k=100, run_path=f'{path}query_{j}/run_07.json')
#     search(i, index_8, model_8, qb, k=100, run_path=f'{path}query_{j}/run_08.json')
#     search(i, index_9, model_9, qb, k=100, run_path=f'{path}query_{j}/run_09.json')
#     search(i, index_10, model_10, qb, k=100, run_path=f'{path}query_{j}/run_10.json')
#     search(i, index_11, model_11, qb, k=100, run_path=f'{path}query_{j}/run_11.json')
#     search(i, index_12, model_12, qb, k=100, run_path=f'{path}query_{j}/run_12.json')
#     search(i, index_13, model_13, qb, k=100, run_path=f'{path}query_{j}/run_13.json')
#     search(i, index_14, model_14, qb, k=100, run_path=f'{path}query_{j}/run_14.json')
#     search(i, index_15, model_15, qb, k=100, run_path=f'{path}query_{j}/run_15.json')
#     search(i, index_16, model_16, qb, k=100, run_path=f'{path}query_{j}/run_16.json')
#     search(i, index_17, model_17, qb, k=100, run_path=f'{path}query_{j}/run_17.json')
#     search(i, index_18, model_18, qb, k=100, run_path=f'{path}query_{j}/run_18.json')
#     search(i, index_19, model_19, qb, k=100, run_path=f'{path}query_{j}/run_19.json')
#     search(i, index_20, model_20, qb, k=100, run_path=f'{path}query_{j}/run_20.json')
#     search(i, index_21, model_21, qb, k=100, run_path=f'{path}query_{j}/run_21.json')
#     search(i, index_22, model_22, qb, k=100, run_path=f'{path}query_{j}/run_22.json')
#     search(i, index_23, model_23, qb, k=100, run_path=f'{path}query_{j}/run_23.json')
#     search(i, index_24, model_24, qb, k=100, run_path=f'{path}query_{j}/run_24.json')
#     search(i, index_25, model_25, qb, k=100, run_path=f'{path}query_{j}/run_25.json')    

## Preprocess all the synthetic searches and turn them into TREC style files

In [ ]:
# List all the JSON outputs of each run

path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_LM/json_format/'
json_files = list_files_from_dir(path, extention='.json')
# json_files

In [ ]:
# We check the size of the sessions
file = "/Users/user/question-retrieval-KIPerWeb/runs/ANN_LM/json_format/query_12/deutsche-telekom_gbert-large-paraphrase-cosine_run_14.json"
assert 100 == len(read_json(file))

## LambdaMart Re-ranker

In [ ]:
def reorder_ranked_list(run, order):
    id2dict = dict((d['docid'], d) for d in run)
    re_ranked = [id2dict[x] for x in order]
    return re_ranked



s_model = SentenceTransformer('LLukas22/paraphrase-multilingual-mpnet-base-v2-embedding-all')


def re_rank(path, model):
    run = read_json(path)
    re_ranked_elements = []
    doc_ids = []
    docs = []
    queries = []
    for i, j in enumerate(run):
        df = pd.Series(j,index=j.keys()).to_frame()
        content = ' '.join(df.T['text'].tolist() + df.T['correct_answers_temp'].tolist()) # -> fix this in the other cross encoder re-ranker
        content = preprocess_documents(content, tagger_de)
        query = df.T['query'].tolist()[0]
#         print(f"Query: {query}\nContent: {content}")
#         print("----------------------------------------")
        doc_ids.append(df.T['docid'].tolist()[0])
        docs.append(content)
        queries.append(query)

    docs = list(zip(doc_ids, queries, docs))
    docs = [(e[0], e[1], e[2], get_cosine_sim(e[1], e[2], s_model)) for e in docs]
#     print(docs)
    origial_order = [e[0] for e in docs]
    
#     print(f"original order: {origial_order}")
    
    scored_data = list(maximize_ndcg(docs).values())[0]
#     print(scored_data)
    scored_data.sort(key=lambda x: x[1], reverse=True)
    re_ranked_list = scored_data
    order = [e[0] for e in re_ranked_list]
#     print(f"reordered {order}")
    re_ordered_list = reorder_ranked_list(run, order)
    path1 = list(Path(path).parts)[-2:]
    path1.insert(1, 'MILP-')
    path1 = tuple(path1)
    path1 = (path1[0], path1[1] + path1[2])
    path1 = '/'.join(path1)
    path2 = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_MILP-Re-ranker/json_format/'
    path3 = path2+path1
    return save_dict_as_json(reorder_ranked_list(run, order), path3)


for e in tqdm(json_files):
    re_rank(e, s_model)

In [ ]:
# List all the JSON outputs of each run

path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_MILP-Re-ranker/json_format/'
json_files = list_files_from_dir(path, extention='.json')
# json_files

In [ ]:
# For each JSON output, transform the results to a TREC format

def to_trec_format(a_path):
    if Path(a_path).suffix == '.json':
        results = read_json(a_path)
        ids = []
        query = []
        similarity = []
        for i,j in enumerate(results):
            ids.append(j['docid'])
            query.append(j['query'])
            try:
                similarity.append(j['cosine_distance'])
            except KeyError:
                similarity.append(j['BM25_score'])
        a_df = pd.DataFrame({'docno':ids, 'qid':query,'score':similarity})
        #         model_name = str('-'.join(Path(e).parts[-1:][0].split('_')[2:-2]))
        model_name = list(Path(a_path).parts)[-1:][0]
        model_name = [e for e in model_name.split('.') if e!='json']
        model_name = ['_'.join(e.split("_")[:-1][:-1]) for e in model_name][0]
        

        a_df['tag'] = model_name
        a_df['Q0'] = 'Q0'
        a_df['rank'] = a_df.index+1
        a_df = a_df[['qid', 'Q0', 'docno', 'rank', 'score', 'tag']]
        
        tsv_path = Path(a_path).with_suffix('.tsv')
        return a_df.to_csv(tsv_path, index = False, sep = '\t', header=False)
    else:
        pass



for e in json_files:
    try:
        to_trec_format(e)
    except IndexError:
        print(f"Error! in {e}")

In [ ]:
#list the TREC formatted files, these have a .tsv extention
list_files_from_dir(path, extention='.tsv')

# Process all the runs and continue with the trec formatting
path_list_processed = list_files_from_dir(path, extention='.tsv')

list_of_dfs = []
for e in path_list_processed:
    df = pd.concat([pd.read_csv(e, header=None, sep='\t', names=["queries", "Q0", 
                                                                 "docid", "rank", "score", "tag"])])
    list_of_dfs.append(df)
    


all_runs = pd.concat(list_of_dfs).reset_index(drop=True)


all_runs= all_runs.merge(queries_file,on='queries')
all_runs = all_runs[["qid", "Q0", "docid", "score", "rank", "tag"]]

all_runs['qid'] = all_runs['qid'].astype(str)
all_runs['docid'] = all_runs['docid'].astype(str)

all_runs

In [ ]:
# Extract runs by different LMs
queries = list(set(all_runs['qid']))
L_models = list(set(all_runs['tag']))
runs = [all_runs.query(f'tag=="{e}"') for e in L_models]

for i, j in zip(runs, L_models):
    i.to_csv(f'/Users/user/question-retrieval-KIPerWeb/runs/ANN_MILP-Re-ranker/trec_format/{j}_run.txt', header = None, sep='\t',index=False )

    

In [ ]:
f = "/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/trec_pools/testbed.csv"

qrels_df = pd.read_csv(f)
qrels_df['qid'] = qrels_df['qid'].astype(str)
qrels_df['docid'] = qrels_df['docid'].astype(str)

qrels = Qrels.from_df(
    df=qrels_df,
    q_id_col="qid",
    doc_id_col="docid",
    score_col="qrel",
)

In [ ]:



path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_MILP-Re-ranker/trec_format/'
trec_runs = list_files_from_dir(path, extention='.txt')
trec_runs

runs = []
for e in trec_runs:
    runs.append(Run.from_file(e))
    
runs_names = []
for e in trec_runs:
    runs_names.append(get_last_element_of_path(e))



In [ ]:
report = create_report(qrels, runs, runs_names, 25,"/Users/user/question-retrieval-KIPerWeb/data/results/ANN_MILP-Re-ranker/ann_MILP_at_25.csv")
report.sort_values(by='ndcg@25',  ascending=False).head(5)

In [ ]:
report = create_report(qrels, runs, runs_names, 50,"/Users/user/question-retrieval-KIPerWeb/data/results/ANN_MILP-Re-ranker/ann_MILP_at_100.csv")
report.sort_values(by='ndcg@50',  ascending=False).head(5)

In [ ]:
report = create_report(qrels, runs, runs_names, 100,"/Users/user/question-retrieval-KIPerWeb/data/results/ANN_MILP-Re-ranker/ann_MILP_at_100.csv")
report.sort_values(by='ndcg@100',  ascending=False).head(5)